In [1]:
from PatchedMeasCal.edge_bfs import CouplingMapGraph
from PatchedMeasCal.tensor_patch_cal import TensorPatchFitter
from PatchedMeasCal.fake_measurement_distributions import renormalise_measurement_results

from PatchedMeasCal.inv_measure_methods import aim, sim

import qiskit
from qiskit.providers.fake_provider import FakeVigo, FakeTokyo

from qiskit.ignis.mitigation.measurement import complete_meas_cal, CompleteMeasFitter
import qiskit.tools.jupyter

In [2]:
n_qubits = 20
n_shots = 32000

backend = FakeTokyo()

# If you want to use the same total number of shots to prepare each calibration
n_shots_qiskit = n_shots / (2 ** n_qubits) # One calibration circuit per bitstring
n_shots_patch = n_shots / (2 * len(backend.configuration().coupling_map)) # 4 for each calibration, but the coupling map double counts
#backend = FakeTokyo()

## Qiskit
This will take approximately forever to complete for larger devices

In [3]:
qr = qiskit.QuantumRegister(n_qubits)
meas_calibs, state_labels = complete_meas_cal(qr=qr, circlabel='mcal')
t_qc = qiskit.transpile(meas_calibs, backend)
cal_results = qiskit.execute(t_qc, backend, shots=n_shots_qiskit).result()
meas_fitter = CompleteMeasFitter(cal_results, state_labels, circlabel='mcal')
meas_filter = meas_fitter.filter

## Patched

In [4]:
tpf = TensorPatchFitter(backend, n_shots=n_shots_patch)
tpf.build(verbose=True)

Building Coupling Graph
Building Edge Calibrations
	Building Calibration Circuits
	Building Edge Patches : [====================] 100.0% 
	Building Edge Patches : [====================] 100.0% 
	Constructing Calibration Circuits : [====================] 100.0% 
	Executing Calibration Circuits
	De-hexing Measurement Results
	Tracing Patched Calibration Results : [====================] 100.0% 
Building Patch Calibrations
	Merging Patches : [====================] 100.0% 
Building Measure Fitter
	Building Meas Fitters from Patches : [====================] 100.0% 


## The circuit to test

In [6]:
circ = qiskit.QuantumCircuit(n_qubits, n_qubits)
initial_layout = list(range(n_qubits))

circ.h(0)
for i in range(1, n_qubits):
    circ.cnot(i - 1, i)

circ.measure(initial_layout, initial_layout)

### Circuit results

In [ ]:
tc = qiskit.transpile(circ, backend=backend, initial_layout=initial_layout, optimization_level=0)
results = qiskit.execute(tc, backend, shots=n_shots, initial_layout=initial_layout, optimization_level=0).result()
bare_res = results.get_counts()

### SIM and AIM

In [8]:
sim_res = sim(circ, backend, n_qubits, n_shots=n_shots, equal_shot_distribution=True)
aim_res = aim(circ, backend, n_qubits, n_shots=n_shots, equal_shot_distribution=True)

### Apply patched

In [12]:
tpf_res= tpf.apply(bare_res)

### Apply qiskit

In [ ]:
#qiskit_res = meas_filter.apply(bare_res)

### Distance Measure

In [13]:
def dist(res, n_shots, n_qubits):
    distance = abs(res['0' * n_qubits] - n_shots / 2) + abs(res['1' * n_qubits] - n_shots / 2)
    distance += sum(abs(res[r]) for r in res if (r != '1' * n_qubits and r != '0' * n_qubits))
    distance /= n_shots
    return distance

### Results

In [17]:
print('Bare', dist(bare_res, n_shots, n_qubits))
print('tpf', dist(tpf_res, n_shots, n_qubits))
#print('qiskit', dist(qiskit_res, n_shots, n_qubits))
print('sim', dist(sim_res, n_shots, n_qubits))
print('aim', dist(aim_res, n_shots // 2, n_qubits))

Bare 0.367875
tpf 0.23239005984249575
qiskit 0.07739274793190992
sim 1.1880625
aim 0.383375


In [31]:
print(bare_res['1' * n_qubits], bare_res['0' * n_qubits])
print(qiskit_res['1' * n_qubits], qiskit_res['0' * n_qubits])
print(tpf_res['1' * n_qubits], tpf_res['0' * n_qubits])

11940 13417
14740.154010047201 15293.54288939345
14106.059186626415 13515.624842031677


In [10]:
sum(qiskit_res.values())

32000.000000003285

In [11]:
sum(tpf_res.values())

31999.999999999996